In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/final-hackathon/sample_submission.csv
/kaggle/input/final-hackathon/Train.csv
/kaggle/input/final-hackathon/Test.csv


In [2]:
train_df = pd.read_csv("/kaggle/input/final-hackathon/Train.csv")
train_df.head()

,CpuName,CpuNumberOfCores,CpuNumberOfThreads,CpuBaseClock,CpuCacheL1,CpuCacheL2,CpuCacheL3,CpuDieSize,CpuFrequency,CpuMultiplier,...,GpuNumberOfShadingUnits,GpuNumberOfTMUs,GpuTextureRate,GpuNumberOfTransistors,GpuVulkan,GameName,GameResolution,GameSetting,Dataset,FPS
0,b'Intel Core i7-7700K',4.0,8.0,100.0,256.0,1024.0,8.0,NaN,4200.0,42.0,...,3584.0,224.0,354400.0,11800.0,b'1.2.131',b'counterStrikeGlobalOffensive',1080.0,b'max',b'userbenchmark',300.0
1,b'AMD Ryzen 7 1800X',8.0,16.0,100.0,768.0,4096.0,16.0,0.000192,3600.0,36.0,...,3584.0,224.0,354400.0,11800.0,b'1.2.131',b'counterStrikeGlobalOffensive',1080.0,b'high',b'userbenchmark',300.0
2,b'Intel Core i7-9700K',8.0,8.0,100.0,512.0,2048.0,12.0,NaN,3600.0,36.0,...,768.0,48.0,66820.0,3300.0,b'1.2.131',b'fortnite',1080.0,b'high',b'userbenchmark',90.0
3,b'Intel Core i5-5200U',2.0,4.0,100.0,128.0,512.0,3.0,NaN,2200.0,22.0,...,192.0,24.0,22800.0,NaN,b'1.1.80',b'counterStrikeGlobalOffensive',720.0,b'low',b'userbenchmark',30.0
4,b'Intel Core i7-5500U',2.0,4.0,100.0,128.0,512.0,4.0,NaN,2400.0,24.0,...,640.0,40.0,44960.0,1870.0,b'1.1.126',b'grandTheftAuto5',720.0,b'low',b'userbenchmark',0.0


In [ ]:
# import pandas as pd
# import sweetviz as sv

# #EDA using Autoviz
# sweet_report = sv.analyze(pd.read_csv("/kaggle/input/final-hackathon/Train.csv"))

# #Saving results to HTML file
# sweet_report.show_html('sweet_report.html')

In [ ]:
# #Install the below libaries before importing
# import pandas as pd
# from ydata_profiling import ProfileReport

# #EDA using pandas-profiling
# profile = ProfileReport(pd.read_csv('/kaggle/input/final-hackathon/Train.csv'), explorative=True)

# #Saving results to a HTML file
# profile.to_file("output.html")

# Observations

1.Two columns has 84% and 97% missing values

2.From EDA we have top 10 features which has highest correlation coefficient with FPS



In [3]:
train_df = train_df.drop('GpuNumberOfExecutionUnits',axis=1)
train_df


,CpuName,CpuNumberOfCores,CpuNumberOfThreads,CpuBaseClock,CpuCacheL1,CpuCacheL2,CpuCacheL3,CpuDieSize,CpuFrequency,CpuMultiplier,...,GpuNumberOfShadingUnits,GpuNumberOfTMUs,GpuTextureRate,GpuNumberOfTransistors,GpuVulkan,GameName,GameResolution,GameSetting,Dataset,FPS
0,b'Intel Core i7-7700K',4.0,8.0,100.0,256.0,1024.0,8.0,NaN,4200.0,42.0,...,3584.0,224.0,354400.0,11800.0,b'1.2.131',b'counterStrikeGlobalOffensive',1080.0,b'max',b'userbenchmark',300.0
1,b'AMD Ryzen 7 1800X',8.0,16.0,100.0,768.0,4096.0,16.0,0.000192,3600.0,36.0,...,3584.0,224.0,354400.0,11800.0,b'1.2.131',b'counterStrikeGlobalOffensive',1080.0,b'high',b'userbenchmark',300.0
2,b'Intel Core i7-9700K',8.0,8.0,100.0,512.0,2048.0,12.0,NaN,3600.0,36.0,...,768.0,48.0,66820.0,3300.0,b'1.2.131',b'fortnite',1080.0,b'high',b'userbenchmark',90.0
3,b'Intel Core i5-5200U',2.0,4.0,100.0,128.0,512.0,3.0,NaN,2200.0,22.0,...,192.0,24.0,22800.0,NaN,b'1.1.80',b'counterStrikeGlobalOffensive',720.0,b'low',b'userbenchmark',30.0
4,b'Intel Core i7-5500U',2.0,4.0,100.0,128.0,512.0,4.0,NaN,2400.0,24.0,...,640.0,40.0,44960.0,1870.0,b'1.1.126',b'grandTheftAuto5',720.0,b'low',b'userbenchmark',0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383244,b'AMD Ryzen 5 3500U',4.0,8.0,100.0,512.0,2048.0,4.0,NaN,2100.0,21.0,...,512.0,32.0,35200.0,4940.0,b'1.2.131',b'counterStrikeGlobalOffensive',720.0,b'med',b'userbenchmark',60.0
383245,b'AMD Ryzen 7 3800X',8.0,16.0,100.0,768.0,4096.0,32.0,0.000074,3900.0,39.0,...,1280.0,80.0,136700.0,4400.0,b'1.2.131',b'worldOfTanks',1080.0,b'max',b'fpsbenchmark',51.0
383246,b'Intel Core i5-7600K',4.0,4.0,100.0,256.0,1024.0,6.0,NaN,3800.0,38.0,...,2560.0,160.0,277300.0,7200.0,b'1.2.131',b'grandTheftAuto5',1080.0,b'max',b'userbenchmark',110.0
383247,b'Intel Core i7-6700K',4.0,8.0,100.0,256.0,1024.0,8.0,NaN,4000.0,40.0,...,1920.0,120.0,202000.0,7200.0,b'1.2.131',b'battlefield1',1080.0,b'max',b'userbenchmark',100.0


In [4]:
test_data = pd.read_csv("/kaggle/input/final-hackathon/Test.csv")
test_data = test_data.drop('GpuNumberOfExecutionUnits',axis=1)
test_data

,Index,CpuName,CpuNumberOfCores,CpuNumberOfThreads,CpuBaseClock,CpuCacheL1,CpuCacheL2,CpuCacheL3,CpuDieSize,CpuFrequency,...,GpuShaderModel,GpuNumberOfShadingUnits,GpuNumberOfTMUs,GpuTextureRate,GpuNumberOfTransistors,GpuVulkan,GameName,GameResolution,GameSetting,Dataset
0,211241,b'Intel Core i3-4130',2.0,4.0,100.0,128.0,512.0,3.0,0.000177,3400.0,...,b'5.1',640.0,40.0,32000.0,1500.0,b'1.2.131',b'playerUnknownsBattlegrounds',1080.0,b'low',b'userbenchmark'
1,412734,b'AMD Ryzen 7 3800X',8.0,16.0,100.0,768.0,4096.0,32.0,0.000074,3900.0,...,b'6.5',1408.0,88.0,157100.0,6600.0,b'1.2.131',b'warframe',1080.0,b'med',b'fpsbenchmark'
2,29630,b'AMD Athlon II X2 260',2.0,2.0,200.0,128.0,1024.0,NaN,0.000117,3200.0,...,b'4',64.0,32.0,20800.0,505.0,b'?',b'counterStrikeGlobalOffensive',1080.0,b'low',b'userbenchmark'
3,311940,b'Intel Core i7-7700K',4.0,8.0,100.0,256.0,1024.0,8.0,NaN,4200.0,...,b'6.4',1920.0,120.0,202000.0,7200.0,b'1.2.131',b'fortnite',1080.0,b'max',b'userbenchmark'
4,333170,b'Intel Core i7-8700K',6.0,12.0,100.0,384.0,1536.0,12.0,NaN,3700.0,...,b'6.4',3584.0,224.0,354400.0,11800.0,b'1.2.131',b'fortnite',1440.0,b'max',b'userbenchmark'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42579,213224,b'Intel Core i5-5200U',2.0,4.0,100.0,128.0,512.0,3.0,NaN,2200.0,...,b'5.1',192.0,24.0,22800.0,NaN,b'1.1.80',b'playerUnknownsBattlegrounds',720.0,b'low',b'userbenchmark'
42580,217148,b'Intel Core i7-8700K',6.0,12.0,100.0,384.0,1536.0,12.0,NaN,3700.0,...,b'6.4',1920.0,120.0,202000.0,7200.0,b'1.2.131',b'playerUnknownsBattlegrounds',1080.0,b'med',b'userbenchmark'
42581,299129,b'Intel Core i5-8400',6.0,6.0,100.0,384.0,1536.0,9.0,NaN,2800.0,...,b'5.1',960.0,80.0,87840.0,2540.0,b'1.1.126',b'fortnite',1080.0,b'high',b'userbenchmark'
42582,296611,b'Intel Core i7-5500U',2.0,4.0,100.0,128.0,512.0,4.0,NaN,2400.0,...,b'5.1',192.0,24.0,22800.0,NaN,b'1.1.80',b'fortnite',720.0,b'med',b'userbenchmark'


In [5]:
print(train_df.shape)

(383249, 44)


In [6]:
train_df.columns

Index(['CpuName', 'CpuNumberOfCores', 'CpuNumberOfThreads', 'CpuBaseClock',
       'CpuCacheL1', 'CpuCacheL2', 'CpuCacheL3', 'CpuDieSize', 'CpuFrequency',
       'CpuMultiplier', 'CpuMultiplierUnlocked', 'CpuProcessSize', 'CpuTDP',
       'CpuNumberOfTransistors', 'CpuTurboClock', 'GpuName', 'GpuArchitecture',
       'GpuBandwidth', 'GpuBaseClock', 'GpuBoostClock', 'GpuBus nterface',
       'GpuNumberOfComputeUnits', 'GpuDieSize', 'GpuDirectX',
       'GpuFP32Performance', 'GpuMemoryBus', 'GpuMemorySize', 'GpuMemoryType',
       'GpuOpenCL', 'GpuOpenGL', 'GpuPixelRate', 'GpuProcessSize',
       'GpuNumberOfROPs', 'GpuShaderModel', 'GpuNumberOfShadingUnits',
       'GpuNumberOfTMUs', 'GpuTextureRate', 'GpuNumberOfTransistors',
       'GpuVulkan', 'GameName', 'GameResolution', 'GameSetting', 'Dataset',
       'FPS'],
      dtype='object')

In [7]:
numeric_columns = train_df.select_dtypes(include=[np.number]).columns
numeric_columns

Index(['CpuNumberOfCores', 'CpuNumberOfThreads', 'CpuBaseClock', 'CpuCacheL1',
       'CpuCacheL2', 'CpuCacheL3', 'CpuDieSize', 'CpuFrequency',
       'CpuMultiplier', 'CpuProcessSize', 'CpuTDP', 'CpuNumberOfTransistors',
       'CpuTurboClock', 'GpuBandwidth', 'GpuBaseClock', 'GpuBoostClock',
       'GpuNumberOfComputeUnits', 'GpuDieSize', 'GpuFP32Performance',
       'GpuMemoryBus', 'GpuMemorySize', 'GpuPixelRate', 'GpuProcessSize',
       'GpuNumberOfROPs', 'GpuNumberOfShadingUnits', 'GpuNumberOfTMUs',
       'GpuTextureRate', 'GpuNumberOfTransistors', 'GameResolution', 'FPS'],
      dtype='object')

In [8]:
non_numeric_columns = train_df.select_dtypes(exclude=[np.number]).columns
non_numeric_columns

Index(['CpuName', 'CpuMultiplierUnlocked', 'GpuName', 'GpuArchitecture',
       'GpuBus nterface', 'GpuDirectX', 'GpuMemoryType', 'GpuOpenCL',
       'GpuOpenGL', 'GpuShaderModel', 'GpuVulkan', 'GameName', 'GameSetting',
       'Dataset'],
      dtype='object')

In [9]:
train_df.describe()

,CpuNumberOfCores,CpuNumberOfThreads,CpuBaseClock,CpuCacheL1,CpuCacheL2,CpuCacheL3,CpuDieSize,CpuFrequency,CpuMultiplier,CpuProcessSize,...,GpuMemorySize,GpuPixelRate,GpuProcessSize,GpuNumberOfROPs,GpuNumberOfShadingUnits,GpuNumberOfTMUs,GpuTextureRate,GpuNumberOfTransistors,GameResolution,FPS
count,383249.000000,383249.000000,383249.00000,383249.000000,383249.000000,378701.000000,201080.000000,383249.000000,383249.000000,383249.000000,...,369397.000000,383249.000000,383249.000000,383249.000000,383196.000000,383249.000000,383249.000000,372904.000000,383249.000000,383249.000000
mean,4.879063,7.894586,108.59394,348.178944,1989.547015,9.625837,0.000194,3495.864986,33.422578,17.298746,...,6033.631946,73314.055397,18.514926,48.482034,1749.957787,111.173618,166697.800114,5993.391763,1065.059844,138.556507
std,1.722696,3.839267,28.15315,168.445787,1805.424633,6.100870,0.000061,445.936519,6.456253,7.031747,...,2708.248274,38601.291937,7.096655,21.610053,907.035277,55.485367,93533.116532,3077.470138,162.578452,87.833997
min,1.000000,1.000000,100.00000,64.000000,512.000000,2.000000,0.000074,1500.000000,8.000000,7.000000,...,128.000000,425.000000,7.000000,1.000000,8.000000,2.000000,850.000000,75.000000,720.000000,0.000000
25%,4.000000,4.000000,100.00000,256.000000,1024.000000,6.000000,0.000177,3300.000000,32.000000,14.000000,...,4000.000000,40510.000000,14.000000,32.000000,1152.000000,80.000000,87840.000000,4313.000000,1080.000000,80.000000
50%,4.000000,8.000000,100.00000,256.000000,1024.000000,8.000000,0.000192,3500.000000,35.000000,14.000000,...,6000.000000,82030.000000,16.000000,48.000000,1792.000000,120.000000,154400.000000,5700.000000,1080.000000,120.000000
75%,6.000000,12.000000,100.00000,384.000000,2048.000000,12.000000,0.000192,3700.000000,37.000000,22.000000,...,8000.000000,107700.000000,16.000000,64.000000,2304.000000,144.000000,202000.000000,7200.000000,1080.000000,180.000000
max,32.000000,64.000000,800.00000,3072.000000,18432.000000,64.000000,0.000435,4700.000000,43.000000,90.000000,...,24000.000000,169900.000000,110.000000,96.000000,5120.000000,320.000000,509800.000000,21100.000000,1440.000000,1000.000000


We have to try several encoding technique to this categorical columns

# Imputation

In [10]:
# handling missing values of numeric columns
imputer = SimpleImputer(strategy='mean')
train_df[numeric_columns] = imputer.fit_transform(train_df[numeric_columns] )

In [11]:
# handling missing values of categorical columns
imputer1 = SimpleImputer(strategy='most_frequent')
train_df[non_numeric_columns] = imputer1.fit_transform(train_df[non_numeric_columns] )

# Approach

1.use catboost or light gbm primarily


2.use target encoding or frequency encoding ,then XGBoost or random forest

# Train-Test Data Split

In [12]:
X = train_df.drop(['FPS'], axis=1)
y = train_df['FPS']

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2,random_state=42)

In [14]:
print(X_train.shape, X_val.shape)

(306599, 43) (76650, 43)


In [15]:
cat_features =['CpuName', 'CpuMultiplierUnlocked', 'GpuName', 'GpuArchitecture',
       'GpuBus nterface', 'GpuDirectX', 'GpuMemoryType', 'GpuOpenCL',
       'GpuOpenGL', 'GpuShaderModel', 'GpuVulkan', 'GameName', 'GameSetting',
       'Dataset']

In [16]:
X_train.describe()

,CpuNumberOfCores,CpuNumberOfThreads,CpuBaseClock,CpuCacheL1,CpuCacheL2,CpuCacheL3,CpuDieSize,CpuFrequency,CpuMultiplier,CpuProcessSize,...,GpuMemoryBus,GpuMemorySize,GpuPixelRate,GpuProcessSize,GpuNumberOfROPs,GpuNumberOfShadingUnits,GpuNumberOfTMUs,GpuTextureRate,GpuNumberOfTransistors,GameResolution
count,306599.000000,306599.000000,306599.000000,306599.000000,306599.000000,306599.000000,306599.000000,306599.000000,306599.000000,306599.000000,...,306599.000000,306599.000000,306599.000000,306599.000000,306599.000000,306599.000000,306599.000000,306599.000000,306599.000000,306599.000000
mean,4.877276,7.892602,108.598120,348.028898,1989.894657,9.622705,0.000194,3495.865557,33.420019,17.302121,...,247.547876,6032.320237,73297.732426,18.524519,48.483589,1749.711708,111.162926,166642.115512,5991.620248,1064.981164
std,1.723099,3.839160,28.087893,168.364000,1807.856467,6.058443,0.000044,446.189346,6.456845,7.037555,...,209.059474,2657.180035,38595.742043,7.113277,21.623258,906.786126,55.470566,93482.966599,3035.210128,162.663067
min,1.000000,1.000000,100.000000,64.000000,512.000000,2.000000,0.000074,1500.000000,8.000000,7.000000,...,32.000000,128.000000,425.000000,7.000000,1.000000,8.000000,2.000000,850.000000,75.000000,720.000000
25%,4.000000,4.000000,100.000000,256.000000,1024.000000,6.000000,0.000192,3300.000000,32.000000,14.000000,...,192.000000,4000.000000,40510.000000,14.000000,32.000000,1152.000000,80.000000,87840.000000,4400.000000,1080.000000
50%,4.000000,8.000000,100.000000,256.000000,1024.000000,8.000000,0.000194,3500.000000,35.000000,14.000000,...,256.000000,6000.000000,82030.000000,16.000000,48.000000,1792.000000,120.000000,154400.000000,5700.000000,1080.000000
75%,6.000000,12.000000,100.000000,384.000000,2048.000000,12.000000,0.000194,3700.000000,37.000000,22.000000,...,256.000000,8000.000000,107700.000000,16.000000,64.000000,2304.000000,144.000000,202000.000000,7200.000000,1080.000000
max,32.000000,64.000000,800.000000,3072.000000,18432.000000,64.000000,0.000435,4700.000000,43.000000,90.000000,...,4096.000000,24000.000000,169900.000000,110.000000,96.000000,5120.000000,320.000000,509800.000000,21100.000000,1440.000000


**As we have 300+ categories in some of the categorical columns we are gonna use these three encodings**


1.Target Encoding

2.Frequency Encoding 

3.Label Encoding

# Encoding

**Target Encoding**

In [ ]:
# import pandas as pd
# from sklearn.model_selection import KFold

# def target_encoding(df, column, target, n_splits=5):
#     # Create a copy of the original dataframe to avoid modifying it
#     df_encoded = df.copy()
    
#     # Initialize the KFold cross-validator
#     kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
#     # Initialize an empty column for the target-encoded values
#     df_encoded[f'{column}_encoded'] = 0
    
#     # Iterate through the KFold splits
#     for train_index, val_index in kf.split(df):
#         # Split into training and validation sets
#         train_data, val_data = df.iloc[train_index], df.iloc[val_index]
        
#         # Calculate the mean target value for each category in the training data
#         means = train_data.groupby(column)[target].mean()
        
#         # Map the means to the validation data
#         df_encoded.loc[val_index, f'{column}_encoded'] = val_data[column].map(means)
    
#     # Handle missing values (if any category wasn't present in the training data)
#     global_mean = df[target].mean()
#     df_encoded[f'{column}_encoded'].fillna(global_mean, inplace=True)
    
#     return df_encoded

# # Example usage:
# data = pd.DataFrame({
#     'category':train_df['CpuName', 'CpuMultiplierUnlocked', 'GpuName', 'GpuArchitecture',
#        'GpuBus nterface', 'GpuDirectX', 'GpuMemoryType', 'GpuOpenCL',
#        'GpuOpenGL', 'GpuShaderModel', 'GpuVulkan', 'GameName', 'GameSetting',
#        'Dataset']
#     'target': train_df['FPS']
# })

# encoded_data = target_encoding(data, 'category', 'target')
# print(encoded_data)

**Frequency Encoding**

In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split

# # Perform frequency encoding for each categorical column
# for col in non_numeric_columns:
#     # Calculate the frequency of each category in the training data
#     freq_encoding = X_train[col].value_counts(normalize=True)
    
#     # Map the frequency back to the categorical column
#     X_train[col + '_freq_enc'] = X_train[col].map(freq_encoding)
    
#     # Apply the same mapping to the test data
#     X_val[col + '_freq_enc'] = X_val[col].map(freq_encoding)
    
#     test_data[col + '_freq_enc'] =test_data[col].map(freq_encoding)

# # Drop the original categorical columns
# X_train.drop(columns=non_numeric_columns, inplace=True)
# X_val.drop(columns=non_numeric_columns, inplace=True)
# test_data.drop(columns=non_numeric_columns, inplace=True)


# # Display the resulting encoded training data
# print("Training Data with Frequency Encoding:\n", X_train)
# print("\nTest Data with Frequency Encoding:\n", X_val)
# print("\nTest Data with Frequency Encoding:\n", test_data)

Target encoding is giving better results but still without any encoding catboost is giving the best results

# Scaling

In [ ]:
# import pandas as pd
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split


# # Initialize the StandardScaler
# scaler = StandardScaler()

# # Fit the scaler on the training data and transform the training data
# X_train_scaled = scaler.fit_transform(X_train)

# # Transform the test data using the same scaler
# X_val_scaled = scaler.transform(X_val)

# # Convert the scaled data back to DataFrame for easy handling (optional)
# X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
# X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns)

# # Display the scaled training and test data
# print("Scaled Training Data:\n", X_train_scaled)
# print("\nScaled Test Data:\n", X_val_scaled)

Without scaling model is giving better results

In [ ]:
# X_train_scaled.describe()

# CatBoost Model

In [ ]:
# from catboost import CatBoostRegressor

# cb =  CatBoostRegressor(iterations = 2000,
#                            random_seed=42,
#                            eval_metric = 'RMSE',
#                            early_stopping_rounds = 50,
#                            learning_rate = 0.05,
#                            depth = 8,
#                            task_type='GPU',  
#                            l2_leaf_reg=7,  
#                            border_count=256,  
#                            subsample=0.85,  
#                            grow_policy='SymmetricTree',  
#                            sampling_frequency = 'PerTree',
#                            bootstrap_type='Bernoulli'
                        
                           

                           
#                        )
# cb.fit(X_train,y_train,cat_features = cat_features,verbose=500,early_stopping_rounds=50)


# XGBoost Model

In [ ]:
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
# import pandas as pd

# # Convert the dataset into DMatrix format (optimized data structure for XGBoost)
# train_dmatrix = xgb.DMatrix(data=X_train, label=y_train)
# test_dmatrix = xgb.DMatrix(data=X_val, label=y_val)

# # Set the parameters for XGBoost
# params = {
#     'objective': 'reg:squarederror',  # For regression tasks
#     'eval_metric': 'rmse',            # Root Mean Squared Error
#     'max_depth': 8,                   # Maximum depth of a tree
#     'eta': 0.05,                       # Learning rate
#     'subsample': 0.7,                 # Subsample ratio of the training instance
#     'colsample_bytree': 0.8,           # Subsample ratio of columns when constructing each tree
#     'seed':42

    
# }

# # Train the XGBoost model
# xg_reg = xgb.train(params, train_dmatrix, num_boost_round=1000, evals=[(test_dmatrix, "Test")], early_stopping_rounds=50)

# # Make predictions
# y_pred = xg_reg.predict(test_dmatrix)

# # Calculate and print RMSE
# rmse = mean_squared_error(y_val, y_pred, squared=False)
# print(f"Root Mean Squared Error (RMSE): {rmse}")

# val_mse = mean_squared_error(y_val, y_pred)
# print(f'Validation MSE: {val_mse}')


for XGboost test_mse around 2487

# Light GBM Model

In [ ]:
# # Load Required Libraries
# import lightgbm as lgb
# import pandas as pd
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.metrics import mean_squared_error
# import matplotlib.pyplot as plt


# # Create LightGBM Dataset
# train_data = lgb.Dataset(X_train, label=y_train)
# val_data = lgb.Dataset(X_val, label=y_val)

# # Define Model Parameters
# params = {
#     'boosting_type': 'gbdt',          # Gradient Boosting Decision Tree
#     'objective': 'regression',        # For regression
#     'metric': 'rmse',                 # Evaluation metric: Root Mean Squared Error
#     'learning_rate': 0.05,
#     'num_leaves': 128,                 # Number of leaves in full tree
#     'max_depth': 8,                  # Depth of tree (-1 means no limit)
#     'n_estimators': 1000              # Number of boosting iterations
# }

# Train the Model (Option 1: Using lgb.train)
# model = lgb.train(
#     params,
#     train_data,
#     valid_sets=[train_data, test_data],
#     num_boost_round=1000,
#     verbose_eval=100
# )

# # Alternatively, Train the Model (Option 2: Using lgb.LGBMRegressor)
# model = lgb.LGBMRegressor(**params)
# model.fit(X_train, y_train, eval_set=[(X_val, y_val)])

# # Make Predictions
# y_pred = model.predict(X_val)

# # Evaluate the Model using RMSE
# rmse = mean_squared_error(y_val, y_pred, squared=False)
# print(f'RMSE: {rmse}')

# # Feature Importance
# lgb.plot_importance(model, max_num_features=10)
# plt.show()


for light gbm test_mse is around 2462

# Deep Learning Model

# predictions

In [ ]:
#  y_pred = cb.predict(X_val)

In [ ]:
# y_pred

In [ ]:
#  val_mse = mean_squared_error(y_val, y_pred)
#  print(f'Validation MSE: {val_mse}')

# Retrain the model for entire data

In [17]:
from catboost import CatBoostRegressor

cb =  CatBoostRegressor(iterations = 2000,
                           random_seed=42,
                           eval_metric = 'RMSE',
                           early_stopping_rounds = 50,
                           learning_rate = 0.05,
                           depth = 8,
                           task_type='GPU',  
                           l2_leaf_reg=7,  
                           border_count=256,  
                           subsample=0.8,  
                           grow_policy='SymmetricTree',  
                           sampling_frequency = 'PerTree',
                           bootstrap_type='Bernoulli'
                        
                           

                           
                       )
cb.fit(X,y,cat_features = cat_features,verbose=500,early_stopping_rounds=50)


0:	learn: 85.2103877	total: 393ms	remaining: 13m 5s
500:	learn: 48.3694632	total: 22.8s	remaining: 1m 8s
1000:	learn: 47.7975879	total: 45.3s	remaining: 45.2s
1500:	learn: 47.4002525	total: 1m 8s	remaining: 22.7s
1999:	learn: 47.0617988	total: 1m 31s	remaining: 0us


In [18]:
test_data.shape

(42584, 44)

In [19]:
Index=test_data['Index']
test_data.drop(['Index'],axis=1,inplace=True)

In [ ]:
#Apply any preprocessing done to Train data on test data as well
#test_data = test_data.select_dtypes(include=[np.number])
#test_data.fillna(test_data.mean(), inplace=True)

In [ ]:
# test1_dmatrix = xgb.DMatrix(data=test_data)

In [20]:
y_test = cb.predict(test_data)

In [21]:
y_test

array([ 48.26614658, 205.89050684,  56.04719226, ...,  80.9356939 ,
        37.61867804, 104.08703908])

In [22]:
result = pd.DataFrame({
    'Index': Index,
    'FPS': y_test
})

In [23]:
result

,Index,FPS
0,211241,48.266147
1,412734,205.890507
2,29630,56.047192
3,311940,123.589955
4,333170,140.075134
...,...,...
42579,213224,34.753298
42580,217148,126.287307
42581,299129,80.935694
42582,296611,37.618678


In [24]:
result.to_csv("submission.csv", index=False) #(for making submission file)